# Easy

## Happy Number

* https://leetcode.com/problems/happy-number/description/
***
* Time Complexity: N/A
    - not easily determined due to how calculating the Happy Number works
    - since you continually sum the square of the digits until you either reach a number you've seen or 1, you cannot determine the time complexity easily
* Space Complexity: O(1)
    - only requires a couple of variables
***
* my initial algorithm was to do follow the steps for finding the Happy Number but keeping track of numbers we've seen using a Set
    - this led to an O(n) space algorithm
* but this can be improved upon by realizing that we're trying to find a cycle and can use the Floyd Cycle Detection algorithm (Tortoise and Hare)
    - to do this, we have a slow pointer and a fast pointer
    - the slow pointer only does a single Happy Number operation once while the fast pointer does it twice
    - if they both end up being equal, you'll know that you've detected a cycle
    - else, if either slow or fast are 1, then you've found the Happy Number

In [ ]:
/**
 * @param {number} n
 * @return {boolean}
 */


// time: # of times it takes to get a happy number or loop
// space: O(time)
var isHappy = function(n) {
    if (n === 1) return true;

    const seen = new Set();

    do {
        seen.add(n);

        // stringify
        const num = `${n}`;

        // for every digit square it and stringify
        let sum = 0;
        for (let i = 0; i < num.length; i++) {
            sum += Math.pow(Number(num[i]), 2);
        }

        // console.log({sum})
        if (sum === 1) return true;

        n = sum;
    } while(!seen.has(n));

    return false;
};

var squareSum = function(n) {
    let sum = 0;
    let temp;

    while (n) {
        // grabs the rightmost digit
        temp = n % 10;

        // adds square of digit
        sum += temp * temp;

        // does a right shift >>
        n = Math.trunc(n / 10);
    }

    return sum;
}

// time: same as above
// space: O(1)
var isHappy = function(n) {
    if (n === 1) return true;
    let slow = n;
    let fast = n;

    do {
        slow = squareSum(slow);
        fast = squareSum(fast);
        fast = squareSum(fast);

        if (slow === 1 || fast === 1) return true;
    } while(slow !== fast);

    return false;
};

# Medium

## Rotate Image

* https://leetcode.com/problems/rotate-image/description/
***
* Time Complexity: O(n$^{2}$)
    - you're traversing around half of the matrix which is (n * n) / 2
* Space Complexity: O(1)
    - only requires space for a couple of variables
***
* we can think of this like peeling an onion layer by layer
* we start off with the outermost layer and work in until we reach the halfway point
    - reason being, if we go any further, we would've already rotated those indices and it would return an incorrect answer
* we also rotate starting from the bottom-left first
    - if we start rotating from the top-left, we would have to keep reassigning a temp variable
    - but if we start from the bottom-left, we would only need a temp variable for the top-left
        * this is a slight optimization
* in order to rotate, we have to do 2 things:
    1. we switch the current column and row. so row = column and column = row
    2. we then calculate a new column using the old row
        * n - r - 1
        * this represents how much the cell is offset from one of the corners
            - e.g. how offset from top-left corner, top-right corner, bottom-right corner, and bottom-left corner respectively

In [1]:
/**
 * @param {number[][]} matrix
 * @return {void} Do not return anything, modify matrix in-place instead.
 */

const rotatePos = (n, r, c) => {
    // n - r - 1 represents how much to offset from one of the corners
    // so if we have [0, 1], we would rotate to [1, 2]
    // [0, 1] is one position away from [0,0], the top-left corner
    // so if we rotate it 90 degrees, [1,2] is 1 position away from [0,2], which is the top-right corner
    return [c, n - r - 1];
}

var rotate = function(matrix) {
    const n = matrix.length;

    // don't have to rotate anything
    if (n === 1) return;

    // you only care about the top half of the rows
    // this is b/c by the time you reach the 2nd half, the matrix is already rotated there
    for (let r = 0; r < ((n - 1) / 2); r++) {
        
        // think of it like the layers of an onion
        // if you rotated the outer layer of an onion first
        // you only care about the inner layers now
        // in this case, n - r - 1 accounts for the outer layers that you've already seen
        // e.g. if you started at layer 1, which is at r = 0, c: range(0, n)
        // then when you look at layer 2, you look at r = 1, c: range(1, n - 0 - 1);
        for (let c = r; c < n - r - 1; c++) {
            let temp = matrix[r][c];
            let [rR, cR] = rotatePos(n ,r, c);
            let [rB, cB] = rotatePos(n, rR, cR);
            let [rL, cL] = rotatePos(n, rB, cB);

            // starts rotating from bottom-left to top-left
            // this is so that only 1 temp variable is needed rather than having to reassign it
            // multiple times
            matrix[r][c] = matrix[rL][cL];
            matrix[rL][cL] = matrix[rB][cB];
            matrix[rB][cB] = matrix[rR][cR];
            matrix[rR][cR] = temp;
        }
    }
};

## Spiral Matrix

* https://leetcode.com/problems/spiral-matrix/description/
***
* Time Complexity: O(n * m), n = numRows, m = numCols
    - have to traverse through every cell in the matrix at most once to add it to the res array
* Space Complexity: O(n * m)
    - res array contains all cells in the matrix but in spiral order
***
* have multiple variables to denote the top, bottom, left, and right boundaries that will be updated
* think of this algorithm like peeling an onion
    - left --> right @ top row
    - top --> bottom @ right col
    - right --> left @ bottom row
    - bottom --> top @ left col
* once we are done with the outer layer of the matrix, we move inwards by updating the variables

In [1]:
/**
 * @param {number[][]} matrix
 * @return {number[]}
 */

var spiralOrder = function(matrix) {
    const res = [];
    const numRows = matrix.length;
    const numCols = matrix[0].length;
    const matrixSize = numRows * numCols;

    let top = 0;
    let bottom = numRows - 1;
    let left = 0;
    let right = numCols - 1;

    while (res.length < matrixSize) {
        
        // left --> right at top row
        for (let c = left; c <= right && res.length < matrixSize; c++) {
            res.push(matrix[top][c]);
        }

        // top --> bottom at rightmost column
        // top + 1 is essentially us moving down a row
        // this is so that we do not count the top-right corner twice

        // this is also why we do bottom - 1
        // we do not want to count the bottom-right corner twice
        // and allows the next for-loop to go from right --> left without +/- 1
        for (let r = top + 1; r <= bottom - 1 && res.length < matrixSize; r++) {
            res.push(matrix[r][right]);
        }

        // right --> left at bottom row
        for (let c = right; c >= left && res.length < matrixSize; c--) {
            res.push(matrix[bottom][c]);
        }

        // bottom --> top at leftmost column
        
        // bottom - 1 so that we do not count the bottom-left corner twice
        // top + 1 so that we do not count the top-left corner twice
        for (let r = bottom - 1; r >= top + 1 && res.length < matrixSize; r--) {
            res.push(matrix[r][left]);
        }

        // similar to Rotate Image
        // once we are done getting the spiral order of the matrix's outer layer
        // we peel it like an onion and move onto the inner layers
        top++;
        bottom--;
        left++;
        right--;
    }

    return res;
}

## Set Matrix Zeroes

* https://leetcode.com/problems/set-matrix-zeroes/description/
***
* Time Complexity: O(n * m), where n = numRows, m = numCols
    - have to traverse the entire matrix to determine which cells have a 0 in them
    - also need to traverse the matrix and 0 out any cells with a 0 in its col/row
* Space Complexity: O(1)
    - we use the matrix itself to keep track of the state of rows/cols that need to be zeroed out
    - we also use a variable, isCol0, to determine if any cells in col 0 need to be zeroed out
***
* we traverse the matrix and take note of which cells have a 0 in them
    - we use the 0th col to mark the rows that we need zeroed out and the 0th row to mark the cols
    - but we also need a variable, isCol0, to determine if col 0 needs to be zeroed out as well
        * reason being, since we use col 0 to keep track of rows, any time we check it, there's ALWAYS going to be a 0 in that column, so it would always be zeroed out by default if we aren't careful
        * therefore, we need that boolean to make sure that col 0 is zeroed out only if there was a 0 in there originally
* once we've done that first traversal, we do a bottom-up traversal of the matrix
    - reason being, we use the top row to keep track of the 0s in each column so it wouldn't make sense to write over it and potentially make the whole matrix into 0s
    - we also only go from [numCols - 1...1] and not to 0 b/c we already have isCol0 to check if col 0 needs to be zeroed out
    - if we check at c = 0 in the loop and we zeroed out a previous row already, then it would automatically zero it out even though it shouldn't have

In [5]:
/**
 * @param {number[][]} matrix
 * @return {void} Do not return anything, modify matrix in-place instead.
 */

 // time: O(m * n)
 // space: O(m + n)
var setZeroes = function(matrix) {
    const numRows = matrix.length;
    const numCols = matrix[0].length;

    const rows = new Set();
    const cols = new Set();

    // traverse through matrix and add rows/cols to their set if matrix[r][c] = 0;
    for (let r = 0; r < numRows; r++) {
        for (let c = 0; c < numCols; c++) {
            if (matrix[r][c] === 0) {
                rows.add(r);
                cols.add(c);
            }
        }
    }

    for (let r = 0; r < numRows; r++) {
        for (let c = 0; c < numCols; c++) {
            if (rows.has(r) || cols.has(c)) {
                matrix[r][c] = 0;
            }
        }
    }
};

 // time: O(m * n)
 // space: O(1)
var setZeroes = function(matrix) {
    const numRows = matrix.length;
    const numCols = matrix[0].length;

    // col = 0 keeps track of the state of the rows
    // so we require this variable to keep track of col0 to make sure
    // that 0s are not assigned here unnecessarily
    // for example, if you have to zero out the bottom row b/c a 0 was at matrix[n - 1][0]
    // THIS DOES NOT MEAN THAT YOU SHOULD ZERO OUT THE COL 0, even though there is a zero there
    // but it's a zero there AFTER you've zeroed out that row
    let isCol0 = false;

    // traverse through matrix and add rows/cols to their set if matrix[r][c] = 0;
    for (let r = 0; r < numRows; r++) {
        // check if the first column has a 0 in it
        if (matrix[r][0] === 0) {
            isCol0 = true;
        }  
        for (let c = 1; c < numCols; c++) {
            if (matrix[r][c] === 0) {
                // leftmost col marks the rows
                matrix[r][0] = 0;

                // topmost row marks the cols
                matrix[0][c] = 0;
            }
        }
    }

    // traverse the matrix from the bottom-up
    // then you want to check if matrix is 0 at row or col level
    for (let r = numRows - 1; r >= 0; r--) {

        // we already have a col0 bool so we do not have to iterate to c = 0
        // if we go from numCols - 1 ... 0, we would actually zero out the col 0 unnecessarily
        // this is b/c if we have recently zeroed out a row at the bottom, then every row
        // at the top will count that 0 even though it was added recently and not part of the
        // original matrix!!!

        // that's why we require the isCol0 bool
        // b/c we do not go to col 0 in this for loop we need some way to set it
        // if a 0 is supposed to be there
        for (let c = numCols - 1; c >= 1; c--) {
            if (matrix[r][0] === 0 || matrix[0][c] === 0) {
                matrix[r][c] = 0;
            }
        }

        if (isCol0) {
            matrix[r][0] = 0; 
        }
    } 
};